In [ ]:
from bigml.api import BigML
from pandas import read_csv
df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv')
df = df.fillna(0) 
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0'
df['Monthlydebt'] = df['DebtRatio'] *  df['MonthlyIncome']
df['LatePayment'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate'] + df['NumberOfTime60-89DaysPastDueNotWorse']
df['NumberOfCredits'] = df['NumberRealEstateLoansOrLines'] + df['NumberOfOpenCreditLinesAndLoans']

df.to_csv('GMSC.csv')
# df[:20]

#Connexion BigML avec seule mention du projet après modification du docker-compose
#environment: 
#    - BIGML_API_KEY=d56301dcac3da169187fc78cf753080cf0922f86
#    - BIGML_USERNAME=CLEMENTGOMBEAUD


# Source
api = BigML(project='project/5d94a3e85a213962e2000304')
source = api.create_source('GMSC.csv')
api.ok(source)

# Dataset
origin_dataset = api.create_dataset(source)
api.ok(origin_dataset)

# Split
train_dataset = api.create_dataset(
    origin_dataset, {"name": "GMSC-Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset, {"name": "GMSC-Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

#Modèle choisit : ensemble
ensemble = api.create_ensemble(train_dataset , {"objective_field" : "SeriousDlqin2yrs"})

evaluation = api.create_evaluation(ensemble, test_dataset)

#Batch prediction 
batch_prediction = api.create_batch_prediction(ensemble, test_dataset,{"all_fields": True,
                                                                        "header": True,
                                                                      "confidence": True,
                                                                      "probabilities":True})
api.ok(batch_prediction)

#fichier csv batch prédiction
api.download_batch_prediction(batch_prediction, filename='BatchPrediction/GMSC_Prediction_Ensemble.csv')

In [ ]:
#Exporter modèle sur Kaggle

df = read_csv('GMSC_Prediction_Ensemble.csv')

from pandas import kaggle_csv
df = kaggle_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv')
df = df.fillna(0) 
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0'
df['Monthlydebt'] = df['DebtRatio'] *  df['MonthlyIncome']
df['LatePayment'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate'] + ['NumberOfTime60-89DaysPastDueNotWorse']
df['NumberOfCredits'] = df['NumberRealEstateLoansOrLines'] + ['NumberOfOpenCreditLinesAndLoans']

# Source
api = BigML(project='project/5d94a3e85a213962e2000304')
source = api.create_source('KaggleGMSC.csv')
api.ok(source)

# Dataset
kaggle_dataset = api.create_dataset(source)
api.ok(kaggle_dataset)

kaggle_test = api.create_batch_prediction(ensemble, kaggle_dataset,{"all_fields": True})
api.ok(kaggle_test)

api.download_batch_prediction(kaggle_test, filename='KaggleTest/GMSCforKaggle.csv')
